In [51]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
from sklearn.preprocessing import LabelEncoder


In [52]:
pd.set_option('display.max_row', 111)
pd.set_option('display.max_column', 111)
plt.style.use('dark_background')

data = pd.read_csv("./KaDo.csv")
df = data.copy()

In [53]:
# Delete unused columns, drop NAs, create additionnal columns, and only keep TICKET_IDS with multiple product which are not the same.
df.drop(columns=['MOIS_VENTE', 'PRIX_NET'], axis=1, inplace=True)
df.dropna(axis='columns')
# df['NB OF RELATIONS'] = 0
# df['RELATED CLI_IDs'] = None
encoder = LabelEncoder()
df['ALL_LIBELLES'] = None
df['CODE_LIBELLE'] = encoder.fit_transform(df['LIBELLE'])
df['CODE_LIBELLE'] = df['CODE_LIBELLE'].apply(lambda x: '_'+str(x))
# Keep only ticket ids with multiple product
df = df.drop_duplicates()
single_tickets = df.duplicated(subset='TICKET_ID', keep=False)
df = df[single_tickets]
# NB: Pour le ticket id 36417517 on peut remarquer qu'en produits achetés ensemble il y a une ressemblance dans le libelle où l'on retrouve LILAS.
# UPDATE : Même chose pour le ticket id 32975823....
# Peut-être intéréssant de concaténer la maille et le libelle et observer les ressemblances ?
# test_df = df[df['LIBELLE'] == 'GD JDM4 GRENADE FL200ML']
# print(test_df)
df.head(30)

,TICKET_ID,FAMILLE,UNIVERS,MAILLE,LIBELLE,CLI_ID,ALL_LIBELLES,CODE_LIBELLE
0,35592159,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 PAMPLEMOUSSE FL 200ML,1490281,None,_730
2,35592159,SOINS DU VISAGE,VIS_CJOUR Jeunes Specifique,VIS_JEUNE_ET_LEVRE,CR JR PARF BIO.SPE AC.SENT.50ML,1490281,None,_200
3,35592159,SOINS DU VISAGE,VIS_DEMAQ AAAR,VIS_AAAR_DEMAQLOTION,EAU MICELLAIRE 3 THES FL200ML,1490281,None,_361
4,35592159,HYGIENE,HYG_DOUCHE JARDINMONDE,HYG_JDM,GD JDM4 TIARE FL 200ML,1490281,None,_732
5,35509899,PARFUMAGE,PARF_EAUX PARFUMS,PARF_PARFUM,EDT UN MATIN AU JARDIN 100ML MUGUET,13290776,None,_415
6,35509899,SOINS DU CORPS,CORPS_LAIT HUILE PLAISIRNATURE,CORPS_HYDR_LAIT_HUILE,LAIT VELOUTE COCO PN2 400ML,13290776,None,_925
7,36417517,HYGIENE,HYG_DOUCHE PARFUMS,HYG_PARFUMEE,GD LILAS FP FL200ML,13290776,None,_733
10,36417517,SOINS DU CORPS,CORPS_LAIT HUILE PARFUMS,CORPS_HYDR_LAIT_HUILE,LAIT LILAS FP FL200ML,13290776,None,_908
12,36417517,PARFUMAGE,PARF_EAUX PARFUMS,PARF_PARFUM,EDT UN MATIN AU JARDIN 100ML LILAS,13290776,None,_463
15,33064616,SOINS DU VISAGE,VIS_CJOUR Jeunes Specifique,VIS_JEUNE_ET_LEVRE,CR JOUR PX/MIX HYDRA/VEG P50ML,20200041,None,_194


In [54]:
# Keep in case
# all_ticket_ids = df['TICKET_ID'].tolist()
# all_libelles_list = []
# for ticket_id in all_ticket_ids:
#     all_libelles = df[df['TICKET_ID'] == ticket_id]['LIBELLE'].tolist()
#     all_libelles_list.append(all_libelles)
# df['ALL_LIBELLE'] = all_libelles_list

In [55]:
# Separate two dataframes, one for calculating the association rules and the other for looking up the libelles(description) to display on our recommendations
orders = df[['TICKET_ID', 'CODE_LIBELLE']]
products = df[['CODE_LIBELLE', 'LIBELLE']].copy()

# Drop duplicated products
products = products[~products.duplicated()]
products = products[~products.duplicated(subset=['CODE_LIBELLE'])]

# Set the index to CODE_LIBELLE
products = products.set_index('CODE_LIBELLE')
# Convert to Series for easier lookups
products = products['LIBELLE']

In [56]:
orders = orders.groupby('TICKET_ID')['CODE_LIBELLE'].apply(list).reset_index()
orders.head()

,TICKET_ID,CODE_LIBELLE
0,32931447,"[_733, _787]"
1,32931448,"[_469, _30, _29]"
2,32931452,"[_706, _1279, _1340]"
3,32931453,"[_253, _971, _687, _1483, _823]"
4,32931454,"[_1164, _1177, _1145]"


In [57]:
# fit the TransactionEncoder and do the transformation
te = TransactionEncoder()
orders_1hot = te.fit(orders['CODE_LIBELLE']).transform(orders['CODE_LIBELLE'])
orders_1hot = pd.DataFrame(orders_1hot, columns=te.columns_)
orders_1hot.head()

,_0,_1,_10,_100,_1000,_1001,_1002,_1003,_1004,_1005,_1006,_1007,_1008,_1009,_101,_1010,_1011,_1012,_1014,_1015,_1016,_1017,_1018,_1019,_102,_1020,_1021,_1022,_1023,_1024,_1025,_1026,_1027,_1028,_1029,_103,_1030,_1031,_1032,_1033,_1034,_1035,_1036,_1037,_1038,_1039,_1040,_1041,_1042,_1043,_1044,_1045,_1046,_1047,_1048,...,_949,_95,_950,_951,_952,_953,_954,_955,_956,_957,_958,_959,_96,_960,_961,_962,_963,_964,_965,_967,_968,_969,_97,_970,_971,_972,_973,_974,_975,_976,_977,_978,_979,_98,_980,_981,_982,_983,_984,_985,_986,_987,_988,_989,_99,_990,_991,_992,_993,_994,_995,_996,_997,_998,_999
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [58]:
# APRIORI METHOD
# ap = apriori(orders_1hot, min_support=0.0001, max_len=10, use_colnames=True)
# ap

In [59]:
%%timeit -n1 -r1

# FPGROWTH METHOD, cf: https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/
fpgrowth(orders_1hot, min_support=0.001, max_len=5, use_colnames=True)

41.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [60]:
fp = fpgrowth(orders_1hot, min_support=0.001, max_len=5, use_colnames=True)
fp

,support,itemsets
0,0.007102,(_733)
1,0.002401,(_29)
2,0.001738,(_469)
3,0.002097,(_1279)
4,0.006841,(_687)
...,...,...
1369,0.001329,"(_859, _707)"
1370,0.001282,"(_859, _270, _707)"
1371,0.001024,"(_1469, _1413)"
1372,0.001124,"(_1363, _1448)"


In [61]:
rules = association_rules(fp, metric="lift", min_threshold=5)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(_733),(_742),0.007102,0.007436,0.001254,0.176562,23.745132,0.001201,1.205391
1,(_742),(_733),0.007436,0.007102,0.001254,0.168648,23.745132,0.001201,1.194317
2,(_733),(_1358),0.007102,0.008315,0.001048,0.147490,17.738715,0.000988,1.163254
3,(_1358),(_733),0.008315,0.007102,0.001048,0.125988,17.738715,0.000988,1.136023
4,(_1344),(_1345),0.014105,0.009621,0.002069,0.146716,15.249597,0.001934,1.160667


In [68]:
def predict(antecedent, rules, max_results=10):
    
    # get the rules for this antecedent
    preds = rules[rules['antecedents'] == antecedent]
    
    # a way to convert a frozen set with one element to string
    preds = preds['consequents'].apply(iter).apply(next)
    
    return preds[:max_results]

In [69]:
preds = predict({'_726'}, rules)
preds

239     _712
1427    _728
1807    _729
1815    _729
1821    _727
1833    _729
1838    _729
1854    _728
1865    _725
1870    _725
Name: consequents, dtype: object

In [72]:
print('Original product :', products['_726'], '\n')

print('Recommended products :')
for stockid in set(preds):  
    print(products[stockid])

Original product : GD JDM4 GRENADE FL200ML 

Recommended products :
GD FL200ML JDM PAMPLEMOUSSE
GD JDM4 ORANGE FL 200ML
GD JDM4 LAVANDIN DE PROVENCE 200ML
GD JDM4 CIT VERT FL 200ML
GD JDM4 LOTUS FL200ML
